# INTRODUCTION
Le but de ce notebook est de relever les anomalies présentes sur les notices d'exemplaire afin de pouvoir exporter des listes à destination des aquéreurs. Le script se compose des parties suivantes :
1. Import de la table `items`
2.Traduction des colonnes pour les rendre lisibles pour les collègues qui auront le tableau sous les yeux 
3. Import des tables de valeurs autorisées
4. Une suite de requêtes sur concernant les différentes colonnes de notre dataframe nommé `items`

# IMPORT DE LA TABLE `items`

In [1]:
import pandas as pd
from datetime import datetime

from kiblib.utils.db import DbConn

In [2]:
db_conn = DbConn().create_engine()

On définit la variable query comme une requête SQL dans laquelle aux champs de la table items on a ajouté le champs itemtype de la table biblioitems:

In [3]:
query = """SELECT i.itemnumber, i.biblionumber, i.biblioitemnumber, i.barcode, i.dateaccessioned, i.booksellerid, i.homebranch, i.price, i.replacementprice, i.replacementpricedate, i.datelastborrowed, i.datelastseen, i.stack, i.notforloan, i.damaged, i.damaged_on, i.itemlost, i.itemlost_on, i.withdrawn, i.withdrawn_on, i.itemcallnumber, i.coded_location_qualifier, i.issues, i.renewals, i.reserves, i.restricted, i.itemnotes, i.itemnotes_nonpublic, i.holdingbranch,i.timestamp, i.location, i.permanent_location, i.onloan, i.cn_source, i.cn_sort, i.ccode, i.materials, i.uri, i.itype, i.more_subfields_xml, i.enumchron, i.copynumber, i.stocknumber, i.new_status, i.exclude_from_local_holds_priority, bi.itemtype
FROM koha_prod.items i
JOIN koha_prod.biblioitems bi ON bi.biblionumber = i.biblionumber """

In [4]:
items = pd.read_sql(query,db_conn)

# MODIFICATION DU DATAFRAME & AMÉLIORATION DE LA LISIBILITÉ

In [5]:
items.columns # avant la traduction

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'barcode',
       'dateaccessioned', 'booksellerid', 'homebranch', 'price',
       'replacementprice', 'replacementpricedate', 'datelastborrowed',
       'datelastseen', 'stack', 'notforloan', 'damaged', 'damaged_on',
       'itemlost', 'itemlost_on', 'withdrawn', 'withdrawn_on',
       'itemcallnumber', 'coded_location_qualifier', 'issues', 'renewals',
       'reserves', 'restricted', 'itemnotes', 'itemnotes_nonpublic',
       'holdingbranch', 'timestamp', 'location', 'permanent_location',
       'onloan', 'cn_source', 'cn_sort', 'ccode', 'materials', 'uri', 'itype',
       'more_subfields_xml', 'enumchron', 'copynumber', 'stocknumber',
       'new_status', 'exclude_from_local_holds_priority', 'itemtype'],
      dtype='object')

In [263]:
items = items.rename(columns={"barcode":"code-barre",
                              "dateaccessioned":"date d'acquisition",
                              "homebranch":"site propriétaire",
                              "holdingbranch":"site dépositaire",
                              "price":"prix",
                              "replacementprice":"coût de remplacement",
                              "datelastborrowed":"date dernier emprunt",
                              "datelastseen":"vu la dernière fois",
                              "damaged_on":"abimé le",
                              "itemlost_on":"perdu le",
                              "withdrawn_on":"retiré de la circulation le",
                              "itemcallnumber":"cote",
                              "issues":"nombre de prêts",
                              "timestamp":"date dernier changement",
                              "permanent_location":"localisation permanente",
                              "onloan":"retour prévu le",
                              "itemnotes":"notes publiques",
                              "itemnotes_nonpublic":"notes internes"})

In [7]:
items.columns #après la traduction

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype'],
      dtype='object')

# Utiliser la jointure de tables pour remplacer les valeurs autorisées

## Notforloan

In [8]:
query2 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'etat'"""

In [9]:
va_notforloan = pd.read_sql(query2, db_conn)

In [10]:
va_notforloan['authorised_value'] = va_notforloan['authorised_value'].astype(int)

In [11]:
items = items.merge(va_notforloan,left_on="notforloan",right_on="authorised_value")
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'authorised_value',
       'lib'],
      dtype='object')

In [12]:
items = items.rename(columns={"authorised_value": "valeurs autorisées", "lib": "statut"})

In [13]:
items = items.drop(columns=['valeurs autorisées'])

In [14]:
query3 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'damaged'"""

In [15]:
va_damaged = pd.read_sql(query3, db_conn)

In [16]:
va_damaged['authorised_value'] = va_damaged['authorised_value'].astype(int)

In [17]:
items = items.merge(va_damaged,left_on="damaged",right_on="authorised_value")

In [18]:
items = items.rename(columns={"lib": "abimé"})

In [19]:
items = items.drop(columns=["authorised_value"])

In [20]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'statut', 'abimé'],
      dtype='object')

## itemlost

In [21]:
query4 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'lost'"""

In [22]:
va_itemlost = pd.read_sql(query4, db_conn)

In [23]:
va_itemlost['authorised_value'] = va_itemlost['authorised_value'].astype(int)

In [24]:
items = items.merge(va_itemlost,left_on="itemlost",right_on="authorised_value")

In [25]:
items = items.rename(columns={"lib": "perdu"})

In [26]:
items = items.drop(columns=["authorised_value"])

In [27]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'statut', 'abimé',
       'perdu'],
      dtype='object')

## Withdrawn

In [28]:
query5 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'retirecoll'"""

In [29]:
va_withdrawn = pd.read_sql(query5, db_conn)

In [30]:
va_withdrawn['authorised_value'] = va_withdrawn['authorised_value'].astype(int)

In [31]:
items = items.merge(va_withdrawn,left_on="withdrawn",right_on="authorised_value")

In [32]:
items = items.rename(columns={"lib": "retiré de la circulation"})

In [33]:
items = items.drop(columns=["authorised_value"])

In [34]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'statut', 'abimé',
       'perdu', 'retiré de la circulation'],
      dtype='object')

## Ccode

In [35]:
query6 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'collection'"""

In [36]:
va_collection = pd.read_sql(query6, db_conn)

In [37]:
items = items.merge(va_collection,left_on="ccode",right_on="authorised_value")

In [38]:
items = items.rename(columns={"lib": "collection"})

In [39]:
items = items.drop(columns=["authorised_value"])

In [40]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'statut', 'abimé',
       'perdu', 'retiré de la circulation', 'collection'],
      dtype='object')

## location

In [41]:
query7 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'loc'"""

In [42]:
va_location = pd.read_sql(query7, db_conn)

In [43]:
items = items.merge(va_location,left_on="location",right_on="authorised_value")

In [44]:
items = items.rename(columns={"lib": "localisation"})

In [45]:
items = items.drop(columns=["authorised_value"])

In [46]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'statut', 'abimé',
       'perdu', 'retiré de la circulation', 'collection', 'localisation'],
      dtype='object')

## itemtype

In [47]:
query8 = """SELECT av.authorised_value,av.lib 
FROM koha_prod.authorised_values av
WHERE category = 'typedoc'"""

In [48]:
va_itemtype = pd.read_sql(query8, db_conn)

In [49]:
items = items.merge(va_itemtype,left_on="itemtype",right_on="authorised_value")

In [50]:
items = items.rename(columns={"lib": "type de support"})

In [51]:
items = items.drop(columns=["authorised_value"])

In [52]:
items.columns

Index(['itemnumber', 'biblionumber', 'biblioitemnumber', 'code-barre',
       'date d'acquisition', 'booksellerid', 'site propriétaire', 'prix',
       'coût de remplacement', 'replacementpricedate', 'date dernier emprunt',
       'vu la dernière fois', 'stack', 'notforloan', 'damaged', 'abimé le',
       'itemlost', 'perdu le', 'withdrawn', 'retiré de la circulation le',
       'cote', 'coded_location_qualifier', 'nombre de prêts', 'renewals',
       'reserves', 'restricted', 'notes publiques', 'notes internes',
       'site dépositaire', 'date dernier changement', 'location',
       'localisation permanente', 'retour prévu le', 'cn_source', 'cn_sort',
       'ccode', 'materials', 'uri', 'itype', 'more_subfields_xml', 'enumchron',
       'copynumber', 'stocknumber', 'new_status',
       'exclude_from_local_holds_priority', 'itemtype', 'statut', 'abimé',
       'perdu', 'retiré de la circulation', 'collection', 'localisation',
       'type de support'],
      dtype='object')

items.columns

* [x] notforloan / statut 
* [x] damaged / abimé
* [x] itemlost / perdu
* [x] withdrawn / retiré de la circulation
* [x] ccode / code collection
* [x] location / localisation


# DÉFINITION DE QUELQUES VARIABLES IMPORTANTES

## Les colonnes à exporter

In [53]:
colonnes_a_exporter = ["code-barre",
                       "date d'acquisition",
                       "site propriétaire",
                       "site dépositaire",
                       "prix",
                       "coût de remplacement",
                       "date dernier emprunt",
                       "vu la dernière fois",
                       "statut",
                       "abimé",
                       "abimé le",
                       "perdu",
                       "perdu le",
                       "retiré de la circulation",
                       "type de support",
                       "collection",
                       "cote",
                       "nombre de prêts",
                       "date dernier changement",
                       "localisation",
                       "localisation permanente",
                       "notes publiques",
                       "notes internes"
                      ]

## Transformation des colonnes au format `date`

In [54]:
items["date d'acquisition"] = pd.to_datetime(items["date d'acquisition"])

In [55]:
items["date dernier emprunt"] = pd.to_datetime(items["date dernier emprunt"])

In [56]:
items["vu la dernière fois"] = pd.to_datetime(items["vu la dernière fois"])

In [57]:
items["abimé le"] = pd.to_datetime(items["abimé le"])

In [58]:
items["perdu le"] = pd.to_datetime(items["perdu le"])

In [59]:
items["date dernier changement"] = pd.to_datetime(items["date dernier changement"])

## Les deadlines

In [60]:
deadline_1_an = pd.to_datetime("today") - pd.Timedelta(365, unit='D')
deadline_1_an

Timestamp('2021-11-19 09:16:30.936651')

In [247]:
deadline_2_ans = pd.to_datetime("today") - pd.Timedelta(365*2, unit='D')

# ANALYSE DES ANOMALIES
## Anomalies sur les données manquantes
### Code-barre manquant

In [154]:
codebarre_ko = items[items["code-barre"].isna()]

In [153]:
codebarre_ko.groupby(["statut","localisation"]).size()

statut             localisation          
1 - En commande    Rez-de-chaussée           39
2 - En traitement  1er étage                  1
                   2e étage -  Jeunesse       8
                   3e étage                   1
                   Magasin collectivités      1
                   Magasin disco 2e étage     1
3 - Empruntable    1er étage                  2
                   2e étage -  Jeunesse       1
                   3e étage                   1
                   Magasin collectivités      1
                   Zèbre                      1
6 -  En retrait    1er étage                  1
dtype: int64

In [157]:
anomalies_docs_codebarre_ko = codebarre_ko[~codebarre_ko["notforloan"].isin([-2,-1])]

In [159]:
anomalies_docs_codebarre_ko.groupby(["statut"]).size()

statut
3 - Empruntable    6
6 -  En retrait    1
dtype: int64

In [160]:
anomalies_docs_codebarre_ko[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_codebarre_ko.xlsx',index=False)

### Site propriétaire manquant

In [162]:
site_proprietaire_ko = items[items["site propriétaire"].isna()]
site_proprietaire_ko

,itemnumber,biblionumber,biblioitemnumber,code-barre,date d'acquisition,booksellerid,site propriétaire,prix,coût de remplacement,replacementpricedate,...,new_status,exclude_from_local_holds_priority,itemtype,statut,abimé,perdu,retiré de la circulation,collection,localisation,type de support


### Prix manquant

Voici une description de la méthode employée pour générer la liste des anomalies d'exemplaires sans prix :
* J'ai défini la variable `prix_ko` comme la liste des exemplaires sans code-barre renseigné
* J'ai ensuite exclu de cette liste les documents du magasin, ainsi que les périodiques (défini par la variable `anomalies_prix_ko_coll`
* Grâce à un `groupby` et `.dt.year`, j'ai généré une liste du total de documents sans prix par an.
* On constate que c'est **à partir de 2020 que les documents sans prix augmentent**
* J'ai donc redéfini ma variable `anomalies_prix_ko_coll` comme la liste des exemplaires acheté il y a moins de 3 ans

> * [ ] Il faudrait voir à filtrer et exclure de la liste les docs du Musée (il y en a que 2) et du Zèbre

In [219]:
prix_ko = items[items["prix"].isna()]

In [209]:
anomalies_prix_ko_coll = prix_ko[~prix_ko.localisation.str.contains('Mag')]
anomalies_prix_ko_coll = anomalies_prix_ko_coll[~anomalies_prix_ko_coll.localisation.str.contains('Pé')]

In [211]:
anomalies_prix_ko_coll.groupby([anomalies_prix_ko["date d'acquisition"].dt.year]).size()

date d'acquisition
2005       2
2006       1
2009       2
2010       2
2011       2
2012       4
2013       3
2014      25
2015      16
2016      91
2017     374
2018     492
2019     986
2020    1516
2021    3093
2022    3458
dtype: int64

In [207]:
anomalies_prix_ko_coll = anomalies_prix_ko_coll[anomalies_prix_ko_coll["date d'acquisition"].dt.year >= 2020]

In [222]:
anomalies_prix_ko_coll.groupby(["localisation","collection"]).size()

localisation  collection                                        
1er étage     AAP - Arts plastiques                                 260
              AAP - Bande dessinée                                   16
              AAP - Dessin animé et animation                         4
              AAP - Dessin animé et animation (documentaires)        23
              AAP - Documentaires sur le cinéma et la télévision    115
                                                                   ... 
Zèbre         JDT - 700 Activités manuelles / Arts                   14
              JDT - 900 Géographie / Histoire                         6
              JRO - Premiers romans                                  31
              JRO - Romans policiers pour enfants                     2
              JRO - Romans pour enfants                              10
Length: 162, dtype: int64

In [218]:
anomalies_prix_ko_coll[colonnes_a_exporter].to_excel("../data/liste_anomalies_exemplaires_prix_ko_coll.xlsx",index=False)

### Côte manquante

In [265]:
cotes_ko = items[items["cote"].isna()]

/tmp/ipykernel_19565/1681895464.py:1: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  cotes_ko = items[items["cote"].isna()]


ValueError: cannot reindex on an axis with duplicate labels

## Anomalies sur les statuts

In [239]:
items.groupby(["notforloan","statut"]).size()

notforloan  statut                                 
-4          5 - En réparation                            1123
-3          6 -  En retrait                              1957
-2          2 - En traitement                            1261
-1          1 - En commande                                39
 0          3 - Empruntable                            215214
 2          4 - Réservé à la consultation sur place     48977
 3          9 - Consultable au musée                       17
 4          8 - Sorti des collections                    8804
 5          7 - En reliure                                105
dtype: int64

### Documents abimés

Voici une description de la méthode employée pour générer la liste des anomalies d'exemplaires abimés : 
* J'ai d'abord affiché une liste des valeurs pour la colonne `abimé` (cela servira pour les requêtes)
* J'ai ensuite défini la variable `docs_abimés` comme la liste des documents marqués en `Abimé`
* J'ai ensuite défini la variable `anomalies_docs_abimes`comme la liste des docs abimés ayant été marqué en abimé depuis plus d'un an (grâce à la variable `deadline_1_an`)
* J'ai ensuit utilisé la fonction `groupby` afin d'afficher la liste des docs abimés par statut
* J'ai pu filtrer et générer deux listes différentes grâce àa la fonction `.localisation.str.contains` : 
 - la première comprends les anomalies localisées ailleurs que dans les magasins `anomalies_docs_abimes_coll`
 - La seconde liste concerne les anomalies localisées dans les différents magasins `anomalies_docs_abimes_mag` 
* **On a comme résultat une liste des documents marqués comme abimés depuis plus d'un an et qui ne sont ni sortis des collections ni des documents localisés au magasin**

In [120]:
items["abimé"].value_counts()

Non               274439
Abimé               3006
Prétendu rendu        52
Name: abimé, dtype: int64

In [121]:
docs_abimes = items[items["abimé"]=="Abimé"]

In [122]:
anomalies_docs_abimes = docs_abimes[docs_abimes["abimé le"]<deadline_1_an]

In [123]:
anomalies_docs_abimes.groupby("statut").size()

statut
2 - En traitement              1
3 - Empruntable              243
5 - En réparation             58
6 -  En retrait                8
8 - Sorti des collections    114
dtype: int64

In [131]:
anomalies_docs_abimes = anomalies_docs_abimes[~anomalies_docs_abimes["notforloan"].isin([4])]

In [132]:
anomalies_docs_abimes_coll = anomalies_docs_abimes[~anomalies_docs_abimes.localisation.str.contains('Mag')]

In [133]:
anomalies_docs_abimes_mag = anomalies_docs_abimes[anomalies_docs_abimes.localisation.str.contains('Magasin')]

In [134]:
anomalies_docs_abimes.groupby(["statut","localisation"]).size()

statut             localisation          
2 - En traitement  1er étage                   1
3 - Empruntable    1er étage                  49
                   2e étage -  Jeunesse       57
                   3e étage                    9
                   Mag Pôle adultes °4         1
                   Mag Pôle adultes °8         1
                   Magasin collectivités     106
                   Magasin disco 3e étage      1
                   Magasin jeunesse            1
                   Rez-de-chaussée             9
                   Zèbre                       9
5 - En réparation  1er étage                   5
                   2e étage -  Jeunesse       29
                   3e étage                    2
                   Mag Pôle adultes BD         1
                   Magasin collectivités       6
                   Rez-de-chaussée             1
                   Zèbre                      14
6 -  En retrait    1er étage                   4
                   2e étage

In [135]:
anomalies_docs_abimes_mag.groupby(["statut","localisation"]).size()

statut             localisation          
3 - Empruntable    Magasin collectivités     106
                   Magasin disco 3e étage      1
                   Magasin jeunesse            1
5 - En réparation  Magasin collectivités       6
dtype: int64

On obtient ici la liste des exemplaires passés en abimé depuis plus d'un an (sont exclus de cette liste les exemplaires sortis des collections).

In [141]:
anomalies_docs_abimes_coll[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_abimes_coll.xlsx',index=False)

In [143]:
anomalies_docs_abimes_coll[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_abimes_mag.xlsx',index=False)

### Documents en retrait

In [228]:
docs_en_retrait = items[items['notforloan'].isin([-3])]
docs_en_retrait_codebarre_ko = docs_en_retrait[docs_en_retrait["code-barre"].isna()]
docs_en_retrait_codebarre_ko

,itemnumber,biblionumber,biblioitemnumber,code-barre,date d'acquisition,booksellerid,site propriétaire,prix,coût de remplacement,replacementpricedate,...,new_status,exclude_from_local_holds_priority,itemtype,statut,abimé,perdu,retiré de la circulation,collection,localisation,type de support
227436,436666,343364,343364,None,2021-12-14,None,MED,NaN,NaN,2021-12-14,...,None,NaN,LS,6 - En retrait,Non,Non,Non,ALT - Livres audio,1er étage,Livre audio


In [226]:
anomalies_docs_en_retrait = docs_en_retrait[docs_en_retrait["date dernier changement"] < deadline_1_an]
anomalies_docs_en_retrait.groupby('localisation').size()

localisation
1er étage                 155
2e étage -  Jeunesse      114
3e étage                   38
Mag Patrimoine fol.         2
Mag Patrimoine °4           2
Mag Patrimoine °8           9
Mag Pôle adultes BD        25
Mag Pôle adultes fol      112
Mag Pôle adultes °4         6
Mag Pôle adultes °8        39
Magasin collectivités     243
Magasin disco 3e étage      3
Magasin jeunesse           24
Rez-de-chaussée            34
Zèbre                       3
dtype: int64

In [229]:
anomalies_docs_en_retrait[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_docs_en_retrait.xlsx',index=False)

### Documents en commande

In [236]:
docs_en_commande = items[items['notforloan'].isin([-1])]

In [232]:
anomalies_docs_en_commande = docs_en_retrait[docs_en_retrait['date dernier changement'] < deadline_1_an]

In [233]:
anomalies_docs_en_commande.groupby(items['location']).size()

location
BUS1A      3
MED0A    243
MED0C     34
MED1A    155
MED2A    114
MED2C     24
MED3A     38
MED3C      3
MED3E     39
MED3F      6
MED3G    112
MED3H     25
MED3I      9
MED3J      2
MED3K      2
dtype: int64

In [242]:
docs_en_reparation = items[items['notforloan'].isin([-4])]
docs_en_reparation.groupby("localisation").size()

localisation
1er étage                  44
2e étage -  Jeunesse      230
3e étage                   32
Mag Patrimoine °8           3
Mag Pôle adultes BD       126
Mag Pôle adultes fol        5
Mag Pôle adultes °8        35
Magasin collectivités     521
Magasin disco 3e étage      1
Magasin jeunesse            2
Rez-de-chaussée            84
Zèbre                      40
dtype: int64

In [250]:
anomalies_docs_en_reparation = docs_en_reparation[docs_en_reparation["date dernier changement"]< deadline_1_an]
anomalies_docs_en_reparation.groupby("localisation").size()

localisation
1er étage                 20
2e étage -  Jeunesse      51
3e étage                   8
Mag Patrimoine °8          2
Mag Pôle adultes BD        2
Mag Pôle adultes fol       2
Mag Pôle adultes °8        2
Magasin collectivités    116
Rez-de-chaussée           10
Zèbre                     18
dtype: int64

In [249]:
# Réfléchir à une deadline pour les Collectivités. Agnès me disait que les filles réparent l'hiver.

In [252]:
anomalies_docs_en_reparation[colonnes_a_exporter].to_excel('../data/liste_anomalies_exemplaires_docs_en_reparation.xlsx',index=False)